# Monte Carlo

Now that you have written your own MD code, you have a recipe for evaluating thermodynamic properties of systems in the microcanonical ensemble. We used MD simulations to sample configuration from the ensemble and, since they all had the same probability, we took their arithmetic mean to calculate ensemble averages. But what would you do if you wanted to sample the canonical ensemble? You would have to evaluate Eq.[](ensemble_av_NVT), in which each configuration has a different weight. Effectively, this also means evaluating very **high-dimensional integrals**, since $\textbf x$ is of dimension $6N$ where $N$ is the number of atoms. 

How could this be done in practice? By sampling $a(\textbf{x}_j)$ on a multi dimenstional grid, multiplying each one by its Boltzmann factor and doing the integration by quadrature. This is very, very expensive and the algorithm scales exponentially with the dimensionality of $\textbf x$. In fact, it is completey prohibitive already for small $N$. Monte carlo is a clever way to evaluate such integrals without resorting to qudrature. Moreover, Monte Carlo is a different approach for sampling equilibrium distribution functions that does not rely on the dynamics in time of the system to generate the configurations.

## Evaluation of high-dimensional integrals

As we have seen, the integrals we usually encounter in statistical mechanics are of the form 

$$
A = \langle a \rangle_f = \int f(\textbf x) \, a(\textbf{x}) \, \mathrm{d} \textbf{x},
$$ 
where 

$$
f(\textbf x) = \frac{1}{\mathcal Z} \mathcal{F}(\mathcal{H} (\textbf{x}))
$$
is a positive, normalized phase space probability density.

We will *assume* that we have an algorithm for obtaining a set of $M$ **independent** samples (configurations of the system) $\textbf x_1 ,..., \textbf x_M$ that are all distributed according to $f(\textbf x)$ with some mean $\langle a \rangle_f$ and variance $\sigma^2 = \langle a^2 \rangle_f - \langle a \rangle_f^2$. This is by no means a trivial task. It is straightforward only for simple distributions and Monte Carlo was designed to address this problem exactly issue. But, for now, we will simply assume such an algorithm exists. If that is the case, we can then evaluate $a(\textbf{x}_j)$ for each sample. 

The central limit theorem (CLT) ensures that if we consider the simple arithmetic mean of the samples as a random variable,

$$
A_M = \frac{1}{M} \sum_{j=1}^{M} a(\textbf{x}_j).
$$
It will be normally distributed with the same mean $ \langle a \rangle_f $ as the samples $ \textbf{x}_j $ and a variance 

$$
\sigma^2_M = \frac{\sigma^2}{M}
$$

In other words, the CLT guarantees that the arithmetic mean of the samples converges to ensemble average of $A$ in the limit $M \to \infty$, because its variance from the mean decays like $M^{-1}$. Note that the CLT does not assume a specific form of the distribution $f(\textbf{x})$. This is very powerful, it means that even when our sampling algorithm (dynamic or not) provides samples from a nonuniform distribution, the arithmetic mean will still converge to the true expectation value. The buttom line? We can use the simple arithmetic mean of the samples for the canonical ensemble, just like in the NVE ensemble, as long as we can generate samples from the desired distribution.

## Random sampling

Now, the question becomes: "How do we generate samples that are distributed according to $f(\textbf{x})$?"

## Importance sampling

## Moves

## Acceptance ration and detailed balance 